### experiment

In [18]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import numpy as np

from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import time

from tqdm import tqdm
from torchsummary import summary
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt

import itertools
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

batch_size = 50
learning_rate = 0.01
num_epoch = 100

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

transform2 = transforms.Compose([
    transforms.Normalize([0.5], [0.5])
])



cuda:0


In [19]:
dataset_font_train = datasets.ImageFolder('Data_extend2/811/train/', transform=transform)
dataset_font_val = datasets.ImageFolder('Data_extend2/811/val/', transform=transform)
dataset_font_test = datasets.ImageFolder('Data_extend2/811/test/', transform=transform)

font_train = torch.utils.data.DataLoader(dataset_font_train, batch_size=64, shuffle=True)
font_val = torch.utils.data.DataLoader(dataset_font_val, batch_size=32)
font_test = torch.utils.data.DataLoader(dataset_font_test, batch_size=32)

In [20]:
def train2(NetModel, times, alpha):
    acc_all = []
    for i in range(times):
        net = NetModel().to(device)
        summary(net, (1, 64, 64))
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5, last_epoch=-1)
        file_pth = 'Model_pth/Loss/'+ str(int(time.time()*100)) + '.pth'

        train_loss_hist = []
        val_loss_hist = []
        min_loss = 10000

        for epoch in range(60):

            net.train()
            running_loss = 0.0
            now_i = 0
            for i, (images, labels) in enumerate(font_train):
                images = images.to(device)
                labels = labels.to(device)

                output1,output2  = net(images)
                loss = criterion(output1, labels) + alpha * criterion(output2, labels)

                optimizer.zero_grad() 
                loss.backward() 
                optimizer.step() 
                theta = 0.0

                running_loss += loss.item()
                # print(i)
                now_i = i
            scheduler.step()


            correct = []
            total = 0.0

            net.eval()
            with torch.no_grad():
                for val_data in font_val:
                    val_image, val_labels = val_data

                    val_image = val_image.to(device)
                    val_labels = val_labels.to(device)

                    output1,output2  = net(images)
                    # loss = criterion(output, labels)

                    val_loss = criterion(output1, labels) + alpha * criterion(output2, labels)
                    #val_loss = criterion(output, val_labels)

                    correct.append(val_loss.item())

            val_loss_tes = sum(correct) / len(correct)

            if val_loss_tes < min_loss:
                min_loss = val_loss_tes
                print("save model")
                torch.save(net.state_dict(), file_pth)

            print('epoch: ',epoch ,'||| train loss : ', round(running_loss / now_i,7), ' <-> test loss : ',  round(val_loss_tes,7), '|||')

        count_test = []

        print('last test:', file_pth)
        net.load_state_dict(torch.load(file_pth))
        net.eval()

        with torch.no_grad():
            # acc = 0
            for test_data in font_test:
                test_images, test_labels = test_data
                
                test_images = test_images.to(device)
                test_labels = test_labels.to(device)

                output1,output2  = net(test_images)
                
                test_outputs = output2.detach().cpu().numpy()
                result = test_outputs.argmax(1)
                
                test_labels = test_labels.detach().cpu().numpy()
                
                count = ((result - test_labels) == 0).sum() / len(result)

                count_test.append(count)
                print(count)
                
            print(sum(count_test) / len(count_test))
            acc_all.append(sum(count_test) / len(count_test))
    print(acc_all)
    print(sum(acc_all) / len(acc_all))
    print(round(sum(acc_all) / len(acc_all), 4))
    return acc_all, sum(acc_all) / len(acc_all), round(sum(acc_all) / len(acc_all), 4)

In [21]:
grid_table = [[0.2, 1], [0.5, 1], [0.8, 1],
              [1, 1],
              [1, 0.2], [1, 0.5], [1, 0.8]]

grid_table2 = [[0.2, 1], [0.5, 1], [0.8, 1],
               [1, 1],
               [1, 0.8], [1, 0.5], [1, 0.2]]

grid_table_b = [5, 2, 1.25, 1, 0.8, 0.5, 0.2]

times = 5

def Train_grid_table(Net, times, grid_table):
    result_grid = []
    for para_alpha,para_beta in grid_table:
        a1, a2, a3 = train2(Net, times, para_alpha, para_beta)
        result_grid.append([para_alpha, para_beta, a1, a2, a3])
        print("----------loss1 weight==" +str(para_alpha)+", loss2 weight==" +str(para_beta)+"----------")
        
    for content in result_grid:
        print("----------loss1 weight==" +str(content[0])+", loss2 weight==" +str(content[1])+"----------")
        print("all loss：" + str(content[2]))
        print("mean loss：     " + str(content[4]) + "    ("+str(content[3])+")   \n ")
        
def Train_grid_table_single(Net, times, grid_table):
    result_grid = []
    for para_alpha in grid_table:
        a1, a2, a3 = train2(Net, times, para_alpha)
        result_grid.append([para_alpha, a1, a2, a3])
        print("----------loss weight==" +str(para_alpha)+"----------")
        
    for content in result_grid:
        print("----------loss weight==" +str(content[0])+"----------")
        print("all loss：" + str(content[1]))
        print("mean loss：     " + str(content[3]) + "    ("+str(content[2])+")   \n ")

In [6]:
class backbone_2loss_cross_self_mask_mask2_Projection(nn.Module):
    def __init__(self):
        super(backbone_2loss_cross_self_mask_mask2_Projection, self).__init__()
        channel_t = 1
        self.conv1 = nn.Sequential(nn.Conv2d(2, 32, 5, stride=1, padding=2),
                                   nn.MaxPool2d(3, 1),
                                   nn.BatchNorm2d(32), nn.ReLU(inplace=True))

        self.groupConv1 = nn.Sequential(nn.Conv2d(32, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True),
                                        nn.Conv2d(32, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True),
                                        nn.Conv2d(32, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True)
                                        )
        self.mask1_a = nn.Sequential(nn.Conv2d(32, channel_t, 1, stride=1, padding=0), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask1_b = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask1_c = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True),
                                     nn.Conv2d(channel_t, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        
        self.mask1_conv = nn.Sequential(nn.Conv2d(channel_t, 32, 1, stride=1), nn.ReLU(inplace=True))
        self.mask1_fc = nn.Linear(56*56, 32)

        self.conv2 = nn.Sequential(nn.Conv2d(32, 32, 5, stride=1, padding=2),
                                   nn.MaxPool2d(3, 1),
                                   nn.BatchNorm2d(32), nn.ReLU(inplace=True))

        self.groupConv2 = nn.Sequential(nn.Conv2d(32, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True),
                                        nn.Conv2d(32, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True)
                                        )

        self.mask2_a = nn.Sequential(nn.Conv2d(32, channel_t, 1, stride=1, padding=0), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask2_b = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask2_c = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True),
                                     nn.Conv2d(channel_t, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        
        self.mask2_conv = nn.Sequential(nn.Conv2d(channel_t, 32, 1, stride=1), nn.ReLU(inplace=True))
        self.mask2_fc = nn.Linear(56*56, 32)

        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 5, stride=1, padding=2),
                                   nn.MaxPool2d(3, 1),
                                   nn.BatchNorm2d(64), nn.ReLU(inplace=True))

        self.groupConv3 = nn.Sequential(nn.Conv2d(64, 32, 3, stride=1),
                                        nn.BatchNorm2d(32), nn.ReLU(inplace=True)
                                        )

        self.mask3_a = nn.Sequential(nn.Conv2d(32, channel_t, 1, stride=1, padding=0), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask3_b = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask3_c = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True),
                                     nn.Conv2d(channel_t, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        
        self.mask3_conv = nn.Sequential(nn.Conv2d(channel_t, 32, 1, stride=1), nn.ReLU(inplace=True))
        self.mask3_fc = nn.Linear(56*56, 32)

        self.conv4 = nn.Sequential(nn.Conv2d(64, 32, 5, stride=1, padding=2),
                                   nn.AvgPool2d(3, 1),
                                   nn.BatchNorm2d(32), nn.ReLU(inplace=True))

        self.mask4_a = nn.Sequential(nn.Conv2d(32, channel_t, 1, stride=1, padding=0), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask4_b = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        self.mask4_c = nn.Sequential(nn.Conv2d(32, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True),
                                     nn.Conv2d(channel_t, channel_t, 3, stride=1, padding=1), 
                                     nn.BatchNorm2d(channel_t), nn.ReLU(inplace=True))
        
        self.mask4_conv = nn.Sequential(nn.Conv2d(channel_t, 32, 1, stride=1), nn.ReLU(inplace=True))
        self.mask4_fc = nn.Linear(56*56, 32)

        self.AdaptiveAvgPool2d = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(32 * 10, 4)
        
        self.fc1 = nn.Linear(32 * 1, 4)

        self.Transpose1 = nn.ConvTranspose2d(128, 64, 5, stride=1, padding=2)
        self.Transpose2 = nn.ConvTranspose2d(64, 32, 5, stride=1, padding=2)
        self.Transpose3 = nn.ConvTranspose2d(32, 32, 5, stride=1, padding=2)

        self.singleConv1 = nn.Sequential(nn.Conv2d(32, 32, (1, 3), stride=1, padding=(0, 1)), nn.ReLU(inplace=True))
        
        self.proj0 = nn.Conv2d(2048, 8192, kernel_size=1, stride=1)
        self.proj1 = nn.Conv2d(2048, 8192, kernel_size=1, stride=1)
        self.proj2 = nn.Conv2d(2048, 8192, kernel_size=1, stride=1)

        
    def forward(self, input):
        
        Projection_x = torch.mean(input, dim=2, keepdim=True).squeeze(1)
        Projection_y = torch.mean(input, dim=3, keepdim=True).squeeze(1)
        #print(Projection_x.shape, Projection_y.shape)
        
        Projection_matrix = torch.bmm(Projection_y, Projection_x).unsqueeze(1)
        #print(Projection_matrix.shape)
        #
        # Projection_x = Projection_x.view(Projection_x.size()[0], 1, -1)
        # Projection_y = Projection_y.view(Projection_y.size()[0], 1, -1)
        
        inputs_ = torch.cat([input, Projection_matrix], dim=1)
        # print(inputs_.shape)
        
        x1 = self.conv1(inputs_)
        # x_f1 = self.AdaptiveAvgPool2d(self.se1(x1))

        x2 = self.conv2(x1)
        # x_f2 = self.AdaptiveAvgPool2d(self.se2(x2))

        x3 = self.conv3(x2)
        # x_f3 = self.AdaptiveAvgPool2d(self.se3(x3))

        x4 = self.conv4(x3)
        # x_f4 = self.AdaptiveAvgPool2d(self.se4(x4))
        x32 = x4

        x1 = self.groupConv1(x1)
        x2 = self.groupConv2(x2)
        x3 = self.groupConv3(x3)
        #print(2, x1.shape, x2.shape, x3.shape, x4.shape)

        # sx1 = self.singleConv1(x1)
        # print(sx1.shape)
        # ---------------------1------------------------
        mask1_a = self.mask1_a(x1)
        mask1_b = self.mask1_b(x1)
        mask1_c = self.mask1_c(x1)
        #mask1 = mask1_a * mask1_b * mask1_c
        mask1 = mask1_a + mask1_b + mask1_c
        
        mask1_bc1 = self.mask1_conv(mask1)
        # mask1_bc2 = self.AdaptiveAvgPool2d(mask1)
        mask1_bc2 = mask1.view(mask1.size(0), -1)
        #print(mask1_bc2.shape)
        mask1_bc2 = self.mask1_fc(mask1_bc2)
        mask1_bc2 = mask1_bc2.unsqueeze(2)
        mask1_bc2 = mask1_bc2.unsqueeze(2)
        
        mask1 = mask1_bc1 * mask1_bc2
        # print(mask1.shape, x1.shape)
        
        # ---------------------2------------------------
        mask2_a = self.mask2_a(x2)
        mask2_b = self.mask2_b(x2)
        mask2_c = self.mask2_c(x2)
        #mask2 = mask2_a * mask2_b * mask2_c
        mask2 = mask2_a + mask2_b + mask2_c
        
        mask2_bc1 = self.mask2_conv(mask2)
        #mask2_bc2 = self.AdaptiveAvgPool2d(mask2)
        mask2_bc2 = mask2.view(mask2.size(0), -1)
        
        mask2_bc2 = self.mask2_fc(mask2_bc2)
        mask2_bc2 = mask2_bc2.unsqueeze(2)
        mask2_bc2 = mask2_bc2.unsqueeze(2)
        
        mask2 = mask2_bc1 * mask2_bc2
        
        # ---------------------3------------------------
        mask3_a = self.mask3_a(x3)
        mask3_b = self.mask3_b(x3)
        mask3_c = self.mask3_c(x3)
        #mask3 = mask3_a * mask3_b * mask3_c
        mask3 = mask3_a + mask3_b + mask3_c
        
        mask3_bc1 = self.mask3_conv(mask3)
        # mask3_bc2 = self.AdaptiveAvgPool2d(mask3)
        mask3_bc2 = mask3.view(mask3.size(0), -1)
        mask3_bc2 = self.mask3_fc(mask3_bc2)
        mask3_bc2 = mask3_bc2.unsqueeze(2)
        mask3_bc2 = mask3_bc2.unsqueeze(2)
        
        mask3 = mask3_bc1 * mask3_bc2
        
        # ---------------------4------------------------
        mask4_a = self.mask4_a(x4)
        mask4_b = self.mask4_b(x4)
        mask4_c = self.mask4_c(x4)
        #mask4 = mask4_a * mask4_b * mask4_c
        mask4 = mask4_a + mask4_b + mask4_c
        
        mask4_bc1 = self.mask4_conv(mask4)
        #mask4_bc2 = self.AdaptiveAvgPool2d(mask4)
        mask4_bc2 = mask4.view(mask4.size(0), -1)
        mask4_bc2 = self.mask4_fc(mask4_bc2)
        mask4_bc2 = mask4_bc2.unsqueeze(2)
        mask4_bc2 = mask4_bc2.unsqueeze(2)
        
        mask4 = mask4_bc1 * mask4_bc2
        

        x1 = x1 * nn.Sigmoid()(mask1)
        x2 = x2 * nn.Sigmoid()(mask2)
        x3 = x3 * nn.Sigmoid()(mask3)
        x4 = x4 * nn.Sigmoid()(mask4)
        
        x = torch.cat(
            (self.AdaptiveAvgPool2d(x1 * x2), self.AdaptiveAvgPool2d(x1 * x3), self.AdaptiveAvgPool2d(x1 * x4),
             self.AdaptiveAvgPool2d(x2 * x3), self.AdaptiveAvgPool2d(x2 * x4),
             self.AdaptiveAvgPool2d(x3 * x4),
             
             self.AdaptiveAvgPool2d(x1 * x1),self.AdaptiveAvgPool2d(x2 * x2),
            self.AdaptiveAvgPool2d(x3 * x3),self.AdaptiveAvgPool2d(x4 * x4)), dim=1)
        
        x = x.view(x.size(0), -1)
        yloss = self.fc(x)
        
        xloss = self.AdaptiveAvgPool2d(x32)
        xloss = xloss.view(xloss.size(0), -1)
        xloss = self.fc1(xloss)

        return xloss,yloss

In [134]:
# [0.947    ,0.9527    ,0.9592    ,0.9573    ,0.9546    ,0.9568    ,0.9505       ]
Train_grid_table_single(backbone_2loss_cross_self_mask_mask2_Projection, times, grid_table_b)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]           1,632
         MaxPool2d-2           [-1, 32, 62, 62]               0
       BatchNorm2d-3           [-1, 32, 62, 62]              64
              ReLU-4           [-1, 32, 62, 62]               0
            Conv2d-5           [-1, 32, 62, 62]          25,632
         MaxPool2d-6           [-1, 32, 60, 60]               0
       BatchNorm2d-7           [-1, 32, 60, 60]              64
              ReLU-8           [-1, 32, 60, 60]               0
            Conv2d-9           [-1, 64, 60, 60]          51,264
        MaxPool2d-10           [-1, 64, 58, 58]               0
      BatchNorm2d-11           [-1, 64, 58, 58]             128
             ReLU-12           [-1, 64, 58, 58]               0
           Conv2d-13           [-1, 32, 58, 58]          51,232
        AvgPool2d-14           [-1, 32,

epoch:  11 ||| train loss :  0.6520414  <-> test loss :  7.5680356 |||
epoch:  12 ||| train loss :  0.4452159  <-> test loss :  3.1351676 |||
epoch:  13 ||| train loss :  0.5106871  <-> test loss :  21.492239 |||
epoch:  14 ||| train loss :  0.5624193  <-> test loss :  2.7411571 |||
epoch:  15 ||| train loss :  0.6203329  <-> test loss :  2.7513337 |||
save model
epoch:  16 ||| train loss :  0.3550853  <-> test loss :  0.9715949 |||
save model
epoch:  17 ||| train loss :  0.2257933  <-> test loss :  0.1969019 |||
epoch:  18 ||| train loss :  0.1792322  <-> test loss :  2.2405472 |||
save model
epoch:  19 ||| train loss :  0.2385068  <-> test loss :  0.1238082 |||
epoch:  20 ||| train loss :  0.3466722  <-> test loss :  6.4223261 |||
epoch:  21 ||| train loss :  0.5998084  <-> test loss :  15.236064 |||
epoch:  22 ||| train loss :  0.3822463  <-> test loss :  0.5073348 |||
epoch:  23 ||| train loss :  0.2338068  <-> test loss :  8.3419867 |||
epoch:  24 ||| train loss :  0.1379391  <-> 

save model
epoch:  0 ||| train loss :  6.5103869  <-> test loss :  9.6464739 |||
save model
epoch:  1 ||| train loss :  4.5780796  <-> test loss :  5.2205906 |||
epoch:  2 ||| train loss :  3.3795834  <-> test loss :  16.4072876 |||
epoch:  3 ||| train loss :  2.7296276  <-> test loss :  17.7369995 |||
epoch:  4 ||| train loss :  2.0990843  <-> test loss :  18.5065422 |||
epoch:  5 ||| train loss :  1.7517311  <-> test loss :  8.5421839 |||
save model
epoch:  6 ||| train loss :  1.5366748  <-> test loss :  3.0918229 |||
epoch:  7 ||| train loss :  1.4266353  <-> test loss :  23.7371502 |||
epoch:  8 ||| train loss :  1.1108221  <-> test loss :  14.486228 |||
epoch:  9 ||| train loss :  0.8944818  <-> test loss :  7.9451666 |||
save model
epoch:  10 ||| train loss :  0.8396324  <-> test loss :  2.7680976 |||
epoch:  11 ||| train loss :  0.5796782  <-> test loss :  3.160449 |||
save model
epoch:  12 ||| train loss :  0.6347147  <-> test loss :  0.6380545 |||
save model
epoch:  13 ||| tra

save model
epoch:  0 ||| train loss :  6.2635428  <-> test loss :  28.2094231 |||
save model
epoch:  1 ||| train loss :  4.1035791  <-> test loss :  6.3004413 |||
save model
epoch:  2 ||| train loss :  3.1506232  <-> test loss :  3.5255625 |||
save model
epoch:  3 ||| train loss :  2.6064501  <-> test loss :  1.5925883 |||
epoch:  4 ||| train loss :  1.925222  <-> test loss :  9.8951397 |||
epoch:  5 ||| train loss :  1.5734376  <-> test loss :  3.1094246 |||
epoch:  6 ||| train loss :  1.3645514  <-> test loss :  1.9571702 |||
save model
epoch:  7 ||| train loss :  1.2181901  <-> test loss :  1.5211744 |||
epoch:  8 ||| train loss :  0.9752839  <-> test loss :  2.2498209 |||
epoch:  9 ||| train loss :  0.7370866  <-> test loss :  2.0582333 |||
epoch:  10 ||| train loss :  0.7278031  <-> test loss :  5.3295937 |||
save model
epoch:  11 ||| train loss :  0.7129903  <-> test loss :  0.6555864 |||
epoch:  12 ||| train loss :  0.4674283  <-> test loss :  1.3788288 |||
save model
epoch:  13

save model
epoch:  0 ||| train loss :  6.2720292  <-> test loss :  7.2959013 |||
save model
epoch:  1 ||| train loss :  4.2902414  <-> test loss :  4.1160216 |||
epoch:  2 ||| train loss :  3.3379725  <-> test loss :  6.1259971 |||
epoch:  3 ||| train loss :  2.7448906  <-> test loss :  10.9796925 |||
epoch:  4 ||| train loss :  2.3105903  <-> test loss :  7.9658036 |||
save model
epoch:  5 ||| train loss :  1.9076424  <-> test loss :  3.2517309 |||
save model
epoch:  6 ||| train loss :  1.5946938  <-> test loss :  2.7288566 |||
epoch:  7 ||| train loss :  1.3465609  <-> test loss :  14.8500471 |||
epoch:  8 ||| train loss :  1.4393446  <-> test loss :  14.0916939 |||
save model
epoch:  9 ||| train loss :  0.9323229  <-> test loss :  1.4889748 |||
save model
epoch:  10 ||| train loss :  0.9110309  <-> test loss :  1.4394221 |||
epoch:  11 ||| train loss :  0.6575924  <-> test loss :  11.4428396 |||
epoch:  12 ||| train loss :  0.6330862  <-> test loss :  4.5689893 |||
epoch:  13 ||| tr

...

save model
epoch:  0 ||| train loss :  1.4282331  <-> test loss :  1.8419417 |||
epoch:  1 ||| train loss :  0.9908162  <-> test loss :  2.1526535 |||
save model
epoch:  2 ||| train loss :  0.7190722  <-> test loss :  1.0856768 |||
save model
epoch:  3 ||| train loss :  0.5545364  <-> test loss :  0.5947067 |||
epoch:  4 ||| train loss :  0.4549768  <-> test loss :  1.4566872 |||
epoch:  5 ||| train loss :  0.3844985  <-> test loss :  2.2528777 |||
epoch:  6 ||| train loss :  0.3123981  <-> test loss :  0.788948 |||
save model
epoch:  7 ||| train loss :  0.260886  <-> test loss :  0.4928114 |||
epoch:  8 ||| train loss :  0.2199453  <-> test loss :  2.8262336 |||
epoch:  9 ||| train loss :  0.2132784  <-> test loss :  0.76407 |||
save model
epoch:  10 ||| train loss :  0.1682845  <-> test loss :  0.2792412 |||
epoch:  11 ||| train loss :  0.1470561  <-> test loss :  2.0110996 |||
save model
epoch:  12 ||| train loss :  0.1236259  <-> test loss :  0.0816006 |||
epoch:  13 ||| train loss

save model
epoch:  0 ||| train loss :  1.4005331  <-> test loss :  1.5183694 |||
epoch:  1 ||| train loss :  0.9914149  <-> test loss :  2.7647412 |||
epoch:  2 ||| train loss :  0.751618  <-> test loss :  1.7808357 |||
epoch:  3 ||| train loss :  0.5904554  <-> test loss :  2.0658092 |||
save model
epoch:  4 ||| train loss :  0.4846947  <-> test loss :  1.0481974 |||
save model
epoch:  5 ||| train loss :  0.4064498  <-> test loss :  0.7513581 |||
epoch:  6 ||| train loss :  0.3412582  <-> test loss :  6.6567354 |||
epoch:  7 ||| train loss :  0.2928799  <-> test loss :  1.9266081 |||
save model
epoch:  8 ||| train loss :  0.2492017  <-> test loss :  0.5411118 |||
epoch:  9 ||| train loss :  0.2065584  <-> test loss :  1.5378683 |||
epoch:  10 ||| train loss :  0.1705405  <-> test loss :  2.8881176 |||
epoch:  11 ||| train loss :  0.1614704  <-> test loss :  2.2240238 |||
epoch:  12 ||| train loss :  0.1280151  <-> test loss :  0.7141647 |||
epoch:  13 ||| train loss :  0.1120009  <-> 

save model
epoch:  0 ||| train loss :  1.4387793  <-> test loss :  1.266096 |||
save model
epoch:  1 ||| train loss :  0.9977015  <-> test loss :  0.8348821 |||
epoch:  2 ||| train loss :  0.7342346  <-> test loss :  1.0624602 |||
save model
epoch:  3 ||| train loss :  0.5722491  <-> test loss :  0.7733858 |||
epoch:  4 ||| train loss :  0.4724235  <-> test loss :  1.0931726 |||
save model
epoch:  5 ||| train loss :  0.3602676  <-> test loss :  0.7128642 |||
epoch:  6 ||| train loss :  0.3200737  <-> test loss :  2.0923228 |||
epoch:  7 ||| train loss :  0.2738369  <-> test loss :  5.7641163 |||
epoch:  8 ||| train loss :  0.2146762  <-> test loss :  3.1752696 |||
epoch:  9 ||| train loss :  0.2185834  <-> test loss :  2.0445638 |||
save model
epoch:  10 ||| train loss :  0.1846613  <-> test loss :  0.3069367 |||
epoch:  11 ||| train loss :  0.1301794  <-> test loss :  0.6327348 |||
epoch:  12 ||| train loss :  0.128316  <-> test loss :  1.9084008 |||
save model
epoch:  13 ||| train lo

save model
epoch:  0 ||| train loss :  1.4704939  <-> test loss :  1.3465067 |||
epoch:  1 ||| train loss :  1.0203622  <-> test loss :  3.2316809 |||
save model
epoch:  2 ||| train loss :  0.7334112  <-> test loss :  0.6857477 |||
epoch:  3 ||| train loss :  0.5502612  <-> test loss :  1.0131292 |||
epoch:  4 ||| train loss :  0.4536849  <-> test loss :  1.27246 |||
save model
epoch:  5 ||| train loss :  0.3581707  <-> test loss :  0.5719081 |||
save model
epoch:  6 ||| train loss :  0.3069571  <-> test loss :  0.3061906 |||
epoch:  7 ||| train loss :  0.2605038  <-> test loss :  0.8685093 |||
epoch:  8 ||| train loss :  0.2301576  <-> test loss :  1.3450066 |||
epoch:  9 ||| train loss :  0.1880141  <-> test loss :  0.3556675 |||
save model
epoch:  10 ||| train loss :  0.1733878  <-> test loss :  0.2654279 |||
save model
epoch:  11 ||| train loss :  0.1130103  <-> test loss :  0.0973528 |||
epoch:  12 ||| train loss :  0.1242192  <-> test loss :  0.3096899 |||
epoch:  13 ||| train lo

save model
epoch:  0 ||| train loss :  1.4596922  <-> test loss :  1.4662007 |||
save model
epoch:  1 ||| train loss :  1.0467463  <-> test loss :  1.2516867 |||
epoch:  2 ||| train loss :  0.7872947  <-> test loss :  4.5939488 |||
save model
epoch:  3 ||| train loss :  0.6268965  <-> test loss :  0.8292468 |||
epoch:  4 ||| train loss :  0.4986021  <-> test loss :  1.1376525 |||
epoch:  5 ||| train loss :  0.4197445  <-> test loss :  1.2390158 |||
epoch:  6 ||| train loss :  0.3456981  <-> test loss :  2.4896193 |||
save model
epoch:  7 ||| train loss :  0.2962552  <-> test loss :  0.3842194 |||
epoch:  8 ||| train loss :  0.243283  <-> test loss :  0.5052556 |||
epoch:  9 ||| train loss :  0.2072103  <-> test loss :  2.0669236 |||
epoch:  10 ||| train loss :  0.1688193  <-> test loss :  1.3873457 |||
epoch:  11 ||| train loss :  0.1373976  <-> test loss :  0.3980291 |||
epoch:  12 ||| train loss :  0.145167  <-> test loss :  0.4942984 |||
epoch:  13 ||| train loss :  0.1123659  <-> t